# Importing Library

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout , Dropout ,Flatten
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

# Data Preparation

In [2]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

DATADIR = 'UCI_HAR_Dataset'

In [3]:
def _read_csv(filename):
    return pd.read_csv(filename , header = None , delim_whitespace = True)

def load_signal(subset):
    signal_data = []
    
    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signal_data.append(
            _read_csv(filename).to_numpy()
        ) 
    return np.transpose(signal_data, (1, 2, 0))


def load_y(subset):
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

def load_data():
    
    X_train, X_test = load_signal('train'), load_signal('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [4]:
Xtrain , Xtest , Ytrain , Ytest = load_data()

In [5]:
print(Xtrain.shape)
print(Ytrain.shape)
print(Xtest.shape)
print(Ytest.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


# LSTM Sequence Modeling

In [6]:
model = Sequential()
model.add(LSTM(32,input_shape = (128,9)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(30 , activation = "relu"))
model.add(Dense(6, activation = "softmax" ))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                990       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 186       
Total params: 6,552
Trainable params: 6,552
Non-trainable params: 0
_________________________________________________________________


# Training

In [10]:
model.compile(optimizer = "adam" , loss= "categorical_crossentropy" , metrics = ["accuracy"])
model.fit(Xtrain , Ytrain , epochs = 15 , batch_size = 20 , validation_data = (Xtest , Ytest))

Epoch 1/15
368/368 [==============================] - 10s 21ms/step - loss: 0.3157 - accuracy: 0.8932 - val_loss: 0.3969 - val_accuracy: 0.8768
Epoch 2/15
368/368 [==============================] - 5s 13ms/step - loss: 0.2473 - accuracy: 0.9163 - val_loss: 0.3680 - val_accuracy: 0.8836
Epoch 3/15
368/368 [==============================] - 5s 13ms/step - loss: 0.2357 - accuracy: 0.9204 - val_loss: 0.5403 - val_accuracy: 0.8147
Epoch 4/15
368/368 [==============================] - 5s 13ms/step - loss: 0.2258 - accuracy: 0.9191 - val_loss: 0.3978 - val_accuracy: 0.8741
Epoch 5/15
368/368 [==============================] - 5s 14ms/step - loss: 0.2035 - accuracy: 0.9291 - val_loss: 0.4385 - val_accuracy: 0.8843
Epoch 6/15
368/368 [==============================] - 5s 14ms/step - loss: 0.3105 - accuracy: 0.8936 - val_loss: 0.4902 - val_accuracy: 0.8765
Epoch 7/15
368/368 [==============================] - 5s 13ms/step - loss: 0.2390 - accuracy: 0.9068 - val_loss: 0.3975 - val_accuracy: 0.887

In [11]:
loss , accuracy = model.evaluate(Xtest, Ytest )

93/93 [==============================] - 1s 7ms/step - loss: 0.4933 - accuracy: 0.9016


In [12]:
print("Loss on test :",loss)
print("Accuracy on test :",round(accuracy*100,2),"%")

Loss on test : 0.49325796961784363
Accuracy on test : 90.16 %
